In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import sparse_dot_topn.sparse_dot_topn as ct
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn


In [11]:
pashto_df = pd.read_json('Pashto_Raverty_full.json').transpose()

In [12]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [45]:
def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    print(sparse_matrix)

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    print(left_side)
    print(right_side)

    for index in range(0, nr_matches):
        print(index)
        print(sparserows)
        print(A)
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]

    return pd.DataFrame({'left_side': left_side,
                         'right_side': right_side,
                         'similairity': similairity})

In [46]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix_clean = vectorizer.fit_transform(pashto_df['normalized'])
tf_idf_matrix_dirty = vectorizer.transform(pd.Series(["Emad"]))

In [47]:
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.0016298294067382812


In [48]:
matches_df = get_matches_df(matches, pashto_df['normalized'], pashto_df['normalized'], top=1)

  (0, 7182)	0.7012272762879324
[None]
[None]
0
[0]
ا            alif
اب             ab
آب             ab
ابا           iba
ابابکر    ababakr
           ...   
هیئت        haiat
یحیيٰ       yahya
یرغ         yragh
یږه        yajzah
یمن          yumn
Name: normalized, Length: 9246, dtype: object


In [49]:
matches_df

,left_side,right_side,similairity
0,alif,madah,0.701227
